<a href="https://colab.research.google.com/github/leoalexandre/mvp_adbp/blob/main/mvp_anp_seismic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP de Análise de Dados e Boas Práticas
Por Leonardo da Silva Alexandre

## 1.   Definição do Problema

O dataset utilizado neste projeto foi retirado do site da Agência Nacional do Petróleo (ANP). A criação veio a partir da tabela de atributos retirada do shapefile de todos os programas sísmicos gerados pelo mercado de óleo e gás brasileiro, somado a aplicação de algumas regras presentes na legislação brasileira gerida pela ANP. A legislação nacional pode ser encontrada dentro dos sites da ANP. O shapefile está alocado mais precisamente em:

https://www.gov.br/anp/pt-br/assuntos/exploracao-e-producao-de-oleo-e-gas/dados-tecnicos/acervo-de-dados

O dataset apresenta atributos relacionados aos programas sísmicos com relação a datas, localização, volume, valor, empresas proprietárias dos dados e período de confidencialidade. Seu objetivo é vislumbrar novos processamentos ou aquisições sísmicas com maior potencial de interesse do mercado a partir da identificação de uma ausência de dados de processamento e suas versões em áreas das bacias sedimentares brasileiras que apresentem maior visibilidade da indústria de óleo e gás.
Informações sobre atributos:

  1. **Vol_GB** – Volume da soma dos arquivos do projeto sísmico;
  2. **UDs** – Valor em UD do projeto sísmico;
  3. **Project** – Projeto sísmico oriundo de um programa sísmico;
  4. **Type_Proc** – Tipo de processamento sísmico realizado no programa sísmico;
  5. **Pub** – Classificação do programa sísmico em público ou confidencial;
  6. **Program** – Programa sísmico criado pela ANP por solicitação da empresa;
  7. **Type_Prog** – Classificação do programa sísmico em Exclusivo (Pré-98), Exclusivo, Não-Exclusivo ou de fomento;
  8. **Dim** – Dimensionalidade do dado sísmico;
  9. **date_start** – Data de início do programa;
  10. **date_end** – Data de término do programa;
  11. **date_pub** – Data na qual o programa se torna público perante a legislação vigente;
  12. **Bloc** – Bloco exploratório onde está localizado o programa sísmico;
  13. **Field** – Campo de exploração onde está localizado o programa sísmico;
  14. **Basin** – Bacia geológica onde está localizado o programa sísmico;
  15. **EAD** – Empresa de Aquisição de Dados responsável pelo programa sísmico;
  16. **Operator** – Empresa Operadora detentora dos direitos sobre os dados do programa sísmico.

In [2]:
# Imports
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as ms # para tratamento de missings
from matplotlib import cm
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
# configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

## 2. Carga de Dados

Para a carga dos dados presentes em um arquivo de extensão csv será utilizado o pacote Pandas (Python Data Analysis Library). O arquivo se encontra armazenado na pasta pública no GitHub e será carregado diretamente do local. Um novo cabeçalho será inserido acompanhando a descrição prévia dos atributos. Após o carregamento do dataset realizado, os dados serão expostos para uma análise inicial.

In [47]:
# Carrega arquivo csv usando Pandas usando uma URL

# Informa a URL de importação do dataset
url = "https://raw.githubusercontent.com/leoalexandre/mvp_adbp/main/dados_anp.csv"

# Informa o cabeçalho das colunas
colunas = ['Vol_GB', 'UDs', 'Project', 'Type_Proc', 'Pub', 'Program', 'Type_Prog', 'Dim', 'date_start','date_end', 'date_pub', 'Bloc', 'Field', 'Basin', 'EAD', 'Operator']

# Lê o arquivo utilizando as colunas informadas
dataset = pd.read_csv(url, names=colunas, skiprows=1, delimiter=',')


In [48]:
dataset.head()

,Vol_GB,UDs,Project,Type_Proc,Pub,Program,Type_Prog,Dim,date_start,date_end,date_pub,Bloc,Field,Basin,EAD,Operator
0,0.016620,1,0007_SERGIPE_ALAGOAS_39,STK FIN,Publico,0007_SERGIPE_ALAGOAS_39,Exclusivo Pre-98,Sismica 2D,1970-05-08,1970-08-23,,,,,Petrobras,NaN
1,NaN,,0007_SERGIPE_ALAGOAS_39,Field Data,Publico,0007_SERGIPE_ALAGOAS_39,Exclusivo Pre-98,Sismica 2D,1970-05-08,1970-08-23,,,,,Petrobras,NaN
2,14.677270,2.935453938,0022_BRASILSPAN01_PSTM,MIG PSTM,Confidencial,0022_BRASILSPAN01,Nao-Exclusivo,Sismica 2D,2007-05-17,2008-06-21,2026-06-21,,,multiplas,GX Technology,NaN
3,18.291451,3.658290235,0022_BRASILSPAN01_PSDM,MIG PSDM,Confidencial,0022_BRASILSPAN01,Nao-Exclusivo,Sismica 2D,2007-05-17,2008-06-21,2026-06-21,,,multiplas,GX Technology,NaN
4,4283.437811,42.83437811,0022_BRASILSPAN01,Field Data,Confidencial,0022_BRASILSPAN01,Nao-Exclusivo,Sismica 2D,2007-05-17,2008-06-21,2026-06-21,,,multiplas,GX Technology,NaN


## 3. Análise dos Dados

### 3.1 Estatísticas Descritivas

Vamos iniciar examinando as dimensões do dataset, suas informações e alguns exemplos de linhas. MUDAR

In [49]:
# Mostra as dimensões do dataset
print(dataset.shape)

(5783, 16)


In [50]:
# Mostra as informações do dataset
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5783 entries, 0 to 5782
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Vol_GB      4643 non-null   float64
 1   UDs         5783 non-null   object 
 2   Project     5783 non-null   object 
 3   Type_Proc   5783 non-null   object 
 4   Pub         5783 non-null   object 
 5   Program     5783 non-null   object 
 6   Type_Prog   5783 non-null   object 
 7   Dim         5783 non-null   object 
 8   date_start  5783 non-null   object 
 9   date_end    5783 non-null   object 
 10  date_pub    5783 non-null   object 
 11  Bloc        5783 non-null   object 
 12  Field       5783 non-null   object 
 13  Basin       5783 non-null   object 
 14  EAD         5783 non-null   object 
 15  Operator    3003 non-null   object 
dtypes: float64(1), object(15)
memory usage: 723.0+ KB
None


In [51]:
# Mostra as 10 primeiras linhas do dataset
dataset.head(10)

,Vol_GB,UDs,Project,Type_Proc,Pub,Program,Type_Prog,Dim,date_start,date_end,date_pub,Bloc,Field,Basin,EAD,Operator
0,0.016620,1,0007_SERGIPE_ALAGOAS_39,STK FIN,Publico,0007_SERGIPE_ALAGOAS_39,Exclusivo Pre-98,Sismica 2D,1970-05-08,1970-08-23,,,,,Petrobras,NaN
1,NaN,,0007_SERGIPE_ALAGOAS_39,Field Data,Publico,0007_SERGIPE_ALAGOAS_39,Exclusivo Pre-98,Sismica 2D,1970-05-08,1970-08-23,,,,,Petrobras,NaN
2,14.677270,2.935453938,0022_BRASILSPAN01_PSTM,MIG PSTM,Confidencial,0022_BRASILSPAN01,Nao-Exclusivo,Sismica 2D,2007-05-17,2008-06-21,2026-06-21,,,multiplas,GX Technology,NaN
3,18.291451,3.658290235,0022_BRASILSPAN01_PSDM,MIG PSDM,Confidencial,0022_BRASILSPAN01,Nao-Exclusivo,Sismica 2D,2007-05-17,2008-06-21,2026-06-21,,,multiplas,GX Technology,NaN
4,4283.437811,42.83437811,0022_BRASILSPAN01,Field Data,Confidencial,0022_BRASILSPAN01,Nao-Exclusivo,Sismica 2D,2007-05-17,2008-06-21,2026-06-21,,,multiplas,GX Technology,NaN
5,3.146006,1,0022_GREATERBRASILSPAN_SUBSET2,PSTM,Confidencial,0022_GREATERBRASILSPAN,Nao-Exclusivo,Sismica 2D,2008-10-03,2012-08-31,2030-08-31,,,multiplas,GX Technology,NaN
6,3.920684,1,0022_GREATERBRASILSPAN_SUBSET2,PSDM,Confidencial,0022_GREATERBRASILSPAN,Nao-Exclusivo,Sismica 2D,2008-10-03,2012-08-31,2030-08-31,,,multiplas,GX Technology,NaN
7,9.820742,1.96414848,0022_GREATERBRASILSPAN_SUBSET1,PSDM,Confidencial,0022_GREATERBRASILSPAN,Nao-Exclusivo,Sismica 2D,2008-10-03,2012-08-31,2030-08-31,,,multiplas,GX Technology,NaN
8,13.980689,2.79613774,0022_GREATERBRASILSPAN_SUBSET5,PSDM,Confidencial,0022_GREATERBRASILSPAN,Nao-Exclusivo,Sismica 2D,2008-10-03,2012-08-31,2030-08-31,,,multiplas,GX Technology,NaN
9,16.461309,3.29226172,0022_GREATERBRASILSPAN_SUBSET4,PSDM,Confidencial,0022_GREATERBRASILSPAN,Nao-Exclusivo,Sismica 2D,2008-10-03,2012-08-31,2030-08-31,,,multiplas,GX Technology,NaN


In [52]:
# Mostra as 10 últimas linhas do dataset
dataset.tail(10)

,Vol_GB,UDs,Project,Type_Proc,Pub,Program,Type_Prog,Dim,date_start,date_end,date_pub,Bloc,Field,Basin,EAD,Operator
5773,1.022351,1,R0392_BC10_LOFS_2016_4D_SC0360_M3,PSDM,Publico,R0392_BC10_LOFS_2016_4D_SC0360,Exclusivo,Sismica 3D,2016-04-28,2016-11-17,2021-11-17,BC-10,ARGONAUTA,Campos,Shell Brasil,Shell Brasil
5774,0.292222,1,R0392_BC10_LOFS_2018_4D_BS,PSDM,Publico,R0392_BC10_LOFS_2018_4D,Exclusivo,Sismica 3D,2018-06-01,2018-06-20,2023-06-20,BC-10,ARGONAUTA,Campos,Shell Brasil,Shell Brasil
5775,0.292222,1,R0392_BC10_LOFS_2018_4D_M3,PSDM,Publico,R0392_BC10_LOFS_2018_4D,Exclusivo,Sismica 3D,2018-06-01,2018-06-20,2023-06-20,BC-10,ARGONAUTA,Campos,Shell Brasil,Shell Brasil
5776,0.292222,1,R0392_BC10_LOFS_2018_4D_M4,PSDM,Publico,R0392_BC10_LOFS_2018_4D,Exclusivo,Sismica 3D,2018-06-01,2018-06-20,2023-06-20,BC-10,ARGONAUTA,Campos,Shell Brasil,Shell Brasil
5777,0.600385,1,R0392_BC10_LOFS_2020_4D_BS,PSDM,Confidencial,R0392_BC10_LOFS_2020_4D,Exclusivo,Sismica 3D,2020-01-24,2020-05-31,2025-05-31,BC-10,ARGONAUTA,Campos,Shell Brasil,Shell Brasil
5778,0.600385,1,R0392_BC10_LOFS_2020_4D_M5,PSDM,Confidencial,R0392_BC10_LOFS_2020_4D,Exclusivo,Sismica 3D,2020-01-24,2020-05-31,2025-05-31,BC-10,ARGONAUTA,Campos,Shell Brasil,Shell Brasil
5779,1.014275,1,R0392_BC10_LOFS_2020_4D_M5_DEPTH_ALIGNED,PSDM,Confidencial,R0392_BC10_LOFS_2020_4D,Exclusivo,Sismica 3D,2020-01-24,2020-05-31,2025-05-31,BC-10,ARGONAUTA,Campos,Shell Brasil,Shell Brasil
5780,1.014275,1,R0392_BC10_LOFS_2020_4D_M5_DEPTH_SHIFT,PSDM,Confidencial,R0392_BC10_LOFS_2020_4D,Exclusivo,Sismica 3D,2020-01-24,2020-05-31,2025-05-31,BC-10,ARGONAUTA,Campos,Shell Brasil,Shell Brasil
5781,1.022351,1,R0392_BC10_LOFS_BASELINE,PSDM,Publico,R0392_BC10_LOFS_BASELINE,Exclusivo,Sismica 3D,2013-10-12,2014-05-30,2019-05-30,BC-10,ARGONAUTA,Campos,Shell Brasil,Shell Brasil
5782,1.022351,1,R0392_BC10_LOFS_BASELINE,PSDM,Publico,R0392_BC10_LOFS_BASELINE,Exclusivo,Sismica 3D,2013-10-12,2014-05-30,2019-05-30,BC-10,ARGONAUTA,Campos,Shell Brasil,Shell Brasil
